In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#convetring tables to numpy arrays for convenient indexation
CsvProb = pd.read_csv("prob_01.csv", header=None).to_numpy()
CsvTable = pd.read_csv("table_01.csv", header=None).to_numpy()

In [ ]:
Prob_M = [m for m in CsvProb[0]] #seqences of P(M)
Prob_k = [k for k in CsvProb[1]] #and P(k)

Prob_C = np.zeros(20)

for i in range(20) :
  tmp = zip(np.where(CsvProb == 0))
  for pair in tmp :
    Prob_C += Prob_k[pair[0]] * Prob_M[pair[1]]

#table for P(M, C)
#C by rows, M by columns
Prob_McC = np.zeros((20, 20))

for c in range(20) :
  for m in range(20) :
    tmp = np.where(CsvTable[:, m] == c)
    for k in tmp :
      Prob_McC[c][m] += Prob_M[m] * Prob_k[k]

In [ ]:
#table for P(M|C)
#the same way
Prob_MlC = np.zeros((20, 20))

for i in range(20) :
  Prob_McC[i] /= Prob_C[i]

In [ ]:
#array is expected to cosist of positive floats
def max_idx_of_array (arr) :
  idx = -1
  max = 0.0
  for i in range(arr.shape[0]) :
    if arr[i] > max :
      max, idx = arr[i], i
  return idx

#table for deterministic desicion function
DT = np.zeros((20, 20))

#we define cells with max P(M|C) for a fixed C as 1
#and all else as 0
for row_MlC, row_DT in zip(Prob_MlC, DT) :
  tmp = max_idx_of_array(row_MlC)
  row_DT[tmp] = 1

In [ ]:
#accuracy testing based on average loss function

#given table DT, P(M|C) and sequence P(C) 
#compute average loss
def avg_loss_det (DT, Prob_MlC, Prob_C) :
  r = 0
  for i in range(20) :
    idx = max_idx_of_array(DT[i]) #index of cell defined with 1
    r += Prob_C[i] * (1 - Prob_MlC[i][idx])
  return r

In [ ]:
#same as max_idx_of_array(arr), but returns a list
#if there is several maxima
def max_idxs_of_array(arr) :
  arrn = np.array(arr)
  maximum = np.amax(arrn)
  return np.where(arrn == maximum)

#build table for stochastic function
ST = np.zeros((20, 20))

for row_MlC, row_ST in zip(Prob_MlC, ST) :
  idxs = max_idxs_of_array(row_MlC)
  m = idxs.shape[0]
  for i in idxs :
    row_ST[i] = 1.0 / m

In [ ]:
#given stochastic table ST and cyphertext C (by its index)
#randomly returns index among maxima, that is, decided open text
def stochastic_desicion(ST, C) :
  idxs = np.where(ST[C] > 0)
  m = idxs.shape[0]
  return idxs[np.random.randint(0, m)]